In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ''
    for page in reader.pages:
        full_text += page.extract_text() + '\n'
    return full_text

# Usage
raw_text = extract_text_from_pdf("Enter PDF file path here")

# Save to file
with open("dsm5_raw.txt", "w", encoding="utf-8") as f:
    f.write(raw_text)


unknown widths : 
[0, IndirectObject(5443, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(5447, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(5451, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3779, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3832, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3782, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3841, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3785, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3832, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3788, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3841, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3852, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3852, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3832, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3860, 0, 1785471725936)]
unknown widths : 
[0, IndirectObject(3866, 0, 1785471725936)]
unknown 

In [25]:
import re

def clean_text(text):
    # Remove multiple newlines
    text = re.sub(r'\n+', '\n', text)
    # Remove unwanted characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII
    text = re.sub(r'\s{2,}', ' ', text)         # Collapse multiple spaces
    return text.strip()

# Usage
with open("dsm5_raw.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

cleaned_text = clean_text(raw_text)

with open("dsm5_cleaned.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)


In [26]:
def chunk_text(text, min_words=200, max_words=500):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        end = i + max_words
        chunk = words[i:end]
        if len(chunk) >= min_words:
            chunks.append(" ".join(chunk))
        i += max_words
    return chunks

# Usage
chunks = chunk_text(cleaned_text)

# Save chunks to file (optional)
with open("dsm5_chunks.txt", "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i+1} ---\n{chunk}\n\n")


In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, show_progress_bar=True)

# Optional: Save embeddings
import numpy as np
np.save("dsm5_embeddings.npy", embeddings)


Batches: 100%|██████████| 30/30 [00:18<00:00,  1.60it/s]


In [29]:
import faiss
import numpy as np

# Load embeddings and the chunks if not in memory
embeddings = np.load("dsm5_embeddings.npy")  # if saved earlier

# Convert to float32 (required by FAISS)
embeddings = np.array(embeddings).astype('float32')

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean distance
index.add(embeddings)

# Save index (optional)
faiss.write_index(index, "dsm5_faiss.index")


In [ ]:
def retrieve_top_chunks(query, model, index, chunks, top_k=5):
    query_embedding = model.encode([query])[0].astype('float32')
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
import google.generativeai as genai

genai.configure(api_key="enter api key here")  # Replace with your actual API key

model = genai.GenerativeModel("gemini-2.0-flash")  #Make sure to use the correct model name according to the API documentation

def generate_with_gemini(question, context):
    prompt = f"""You are a helpful assistant with access to DSM-5 knowledge.

Answer the following question based on the context below.

Context:
{context}

Question:
{question}

Answer:"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None


In [49]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
query = "what is the difference between mental disorder and perosnality disorder"
top_chunks = retrieve_top_chunks(query, embed_model, index, chunks)
context = "\n\n".join(top_chunks)

answer = generate_with_gemini(query, context)
if answer:
    print(answer)
else:
    print("Failed to generate answer")


While both mental disorders and personality disorders fall under the umbrella of mental health conditions, there are key distinctions:

*   **Mental Disorder:** This is a broader category that encompasses a wide range of conditions affecting mood, thinking, and behavior. These disorders often involve specific episodes or periods of disturbance that may or may not be persistent.

*   **Personality Disorder:** These disorders are characterized by enduring, inflexible patterns of thinking, feeling, and behaving that deviate markedly from the expectations of the individual's culture. These patterns are pervasive and cause distress or impairment in social, occupational, or other important areas of functioning. They are generally long-standing and present from early adulthood.

In short, mental disorders can be more episodic, while personality disorders involve ingrained, long-term patterns of behavior and inner experience.
